# Trabajo de simulación 3

Para el último trabajo de simulación, vamos a hacer un análisis de datos a partir de la encuesta de salarios de Sysarmy del primer trimestre de 2025 (si están publicados los del segundo trimestre o posterior, podemos usar esos, pero a agosto de 2025 todavía no estaban).

Los datos oficiales se encuentran [acá](https://docs.google.com/spreadsheets/d/1hlLwv9SLJvrnsTq_UsEAHkHGNiziH7IdT1lJd4fq6kU/edit?gid=1462536742#gid=1462536742). Por si en algún momento el enlace se cae o cambia de ubicación, una versión alojada en mi drive puede encontrarse [acá](https://docs.google.com/spreadsheets/d/1Vq9F6xE03fR0x6pXlRMDN_bww7NWItjMW0U89qV3AvM/edit?usp=sharing).

Las consignas de este trabajo no son tan dirigidas como las de los trabajos anteriores, pues en el análisis de datos, siempre hay libertad y margen para la creatividad y la producción personal. Sin embargo, les compartimos algunas pautas de lo que debe tener, como mínimo, este trabajo.

Pautas generales y **OBLIGATORIAS** para la aprobación de la entrega:

*   Debe replicarse, como mínimo, un análisis similar al aquí presentado, para estos datos (SysArmy 2025 o cualquier otro dataset que sea de su interés). Por "replicar" nos referimos a que el análisis debe incluir: inspección y limpieza de los datos, descripción y visualización, o estudio de alguna variable de interés a partir de alguna hipótesis o conjetura.

*   Debe escribirse en formato "informe", es decir, no se trata de exhibir código y gráficos, sino de explicar qué se observa y por qué es relevante observar eso. El informe es requisito **excluyente**. No se aprueba el trabajo de simulación sin él. Este informe breve debe entregarse en pdf, en esta entrega **NO** se evalúa el *colab*, sino el reporte. No tiene que ser largo, al contrario, tiene que ser de calidad. Como dice el dicho: lo bueno -si breve- dos veces bueno...

*   El trabajo debe contener, como mínimo, **una conjetura que sea sometida a prueba y de la que se exhiba alguna conclusión fundamentada**, como se hizo en el caso de los datos de 2020 para el salario medio bruto por género y para hombres y mujeres con nivel universitario completo. Por ejemplo, frente a la pregunta de si el salario medio de mujeres y hombres es igual, podríamos poner en práctica lo que estudiamos sobre convergencia para, de alguna forma, darnos una idea de cuán probable es observar lo que efectivamente estamos observando. Este es un "coqueteo" con la estadística inferencial, que no estudiamos formalmente en la materia, pero que es válido comenzar a encarar con todo lo que hemos estudiado. Esta conjetura puede hacerse con datos propios, si es que eligen trabajar con otro dataset.

El resto de la producción queda a criterio de los grupos. Esperamos que haya un interés genuino en tratar de extraer información a partir de estos datos. ¡Muchos éxitos!

**PD: El formato "informe" puede ser cambiado por el formato "póster/infografía" si es que así prefieren.**

# Optimización salarial

Para realizar nuestro análisis tomamos la perspectiva de un profesional del sector IT que necesita transfomar su carrera por alguna razón. Sabemos que esta industria se caracteriza por un nivel de rotación alto y los recorridos profesionales suelen ser muy variados. La pregunta central de nuestro trabajo es _¿cómo puedo maximizar mi salario?_, es decir, _¿cuál es la **combinación ganadora** que se podría obtener como insight de este dataset?_ Los resultados obtenidos podrían ser de utilidad tanto para el _junior_, que necesita alguna "brújula" para ver por dónde orientarse dentro del mercado laboral, como para el _senior_ que quizá se siente estancado y necesite reorientar su desarrollo profesional dentro del sector. 

In [ ]:
import pandas as pd
import numpy as np
import warnings
import re
warnings.filterwarnings('ignore')

# Visualización
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score

# Configuración de Plotly
pio.templates.default = "plotly_white"

## Inspección y limpieza de datos

Comenzamos importando las librerías necesarias y cargando el dataset. Luego inspeccionamos los datos para ver qué variables tenemos disponibles y qué tipo de datos contienen. 

In [ ]:
file_path = 'data/Sysarmy_sueldos_2025.01CLEAN.csv'

try:
    df = pd.read_csv(file_path, encoding='utf-8')
    print(f"*** Archivo cargado exitosamente")
    print(f"    Dimensiones: {df.shape[0]} filas x {df.shape[1]} columnas\n")
except FileNotFoundError:
    print(f"ERROR: Archivo no encontrado en '{file_path}'")
    df = None
except Exception as e:
    print(f"ERROR: {e}")
    df = None

if df is None:
    raise Exception("No se pudo cargar el dataset")


Luego procesamos los datos para dejarlos en un formato adecuado para el análisis. Esto incluye manejar valores faltantes, convertir variables categóricas en numéricas (si es necesario) y asegurarnos de que todas las variables estén en el tipo de dato correcto.

In [ ]:
# Renombrar columnas
column_mapping = {
    'donde_estas_trabajando': 'provincia',
    'dedicacion': 'dedicacion',
    'tipo_de_contrato': 'tipo_contrato',
    'ultimo_salario_mensual_o_retiro_bruto_en_pesos_argentinos': 'salario_bruto_ars',
    'ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos': 'salario_neto_ars',
    'pagos_en_dolares': 'forma_pago',
    'trabajo_de': 'puesto',
    'anos_de_experiencia': 'experiencia',
    'antiguedad_en_la_empresa_actual': 'antiguedad',
    'modalidad_de_trabajo': 'modalidad',
    'cantidad_de_personas_en_tu_organizacion': 'tamano_empresa',
    'tengo_edad': 'edad',
    'genero': 'genero',
    'seniority': 'seniority',
    '_sal': 'salario_ars',
    'cuantas_personas_tenes_a_cargo': 'personas_a_cargo',
    'si_tu_sueldo_esta_dolarizado_cual_fue_el_ultimo_valor_del_dolar_que_tomaron': 'tc_reportado'
}

df_clean = df.rename(columns=column_mapping)

# Convertir a numéricos
df_clean['salario_bruto_ars'] = pd.to_numeric(df_clean['salario_bruto_ars'], errors='coerce')
df_clean['salario_ars'] = pd.to_numeric(df_clean['salario_ars'], errors='coerce')
df_clean['tc_reportado'] = pd.to_numeric(df_clean['tc_reportado'], errors='coerce')

# Calcular salario en USD
print("[PASO 1] Calculando salarios en USD con TC individual...\n")

# Inicializar columna
df_clean['salario_usd'] = np.nan

# Prioridad 1: Usar TC reportado por cada persona
mask_con_tc = (df_clean['tc_reportado'].notna() & 
               (df_clean['tc_reportado'] > 0) & 
               df_clean['salario_ars'].notna())

df_clean.loc[mask_con_tc, 'salario_usd'] = (
    df_clean.loc[mask_con_tc, 'salario_ars'] / df_clean.loc[mask_con_tc, 'tc_reportado']
)

registros_con_tc = mask_con_tc.sum()
print(f"    * {registros_con_tc} registros con TC reportado individual")

# Prioridad 2: Calcular TC mediano para los restantes
tc_mediano = df_clean.loc[df_clean['tc_reportado'] > 0, 'tc_reportado'].median()
print(f"    * TC mediano de la encuesta: ${tc_mediano:,.2f}")

# Aplicar TC mediano a los que no tienen TC reportado
mask_sin_tc = df_clean['salario_usd'].isna() & df_clean['salario_ars'].notna()
if mask_sin_tc.sum() > 0:
    df_clean.loc[mask_sin_tc, 'salario_usd'] = (
        df_clean.loc[mask_sin_tc, 'salario_ars'] / tc_mediano
    )
    print(f"    * {mask_sin_tc.sum()} registros usando TC mediano\n")

# Crear categorías
df_clean['es_caba'] = df_clean['provincia'].apply(
    lambda x: 'CABA' if x == 'Ciudad Autonoma de Buenos Aires' else 'Resto del pais'
)

df_clean['tiene_equipo'] = df_clean['personas_a_cargo'].apply(
    lambda x: 'Con equipo' if pd.notna(x) and x > 0 else 'Sin equipo'
)

print(f"Dataset preparado: {df_clean.shape}")
print(f"Registros con salario USD: {df_clean['salario_usd'].notna().sum()}\n")

Luego de todo el procesamiento "de rigor" que exige un análisis de datos, nos queda un dataset limpio y listo para analizar. En este punto comenzamos con la búsqueda de "la combinación ganadora", es decir, la combinación de variables que maximiza el salario.

In [ ]:
# Dataset para análisis
columnas_analisis = ['salario_usd', 'experiencia', 'es_caba', 'dedicacion',
                     'tipo_contrato', 'puesto', 'seniority', 'modalidad',
                     'edad', 'antiguedad', 'personas_a_cargo', 'forma_pago', 
                     'genero', 'provincia', 'tamano_empresa', 'tiene_equipo']

columnas_analisis = [col for col in columnas_analisis if col in df_clean.columns]
df_analisis = df_clean[columnas_analisis].dropna(subset=['salario_usd'])

print(f"[INFO] Variables analizadas: {len(columnas_analisis)}")
print(f"[INFO] Registros validos: {len(df_analisis)}\n")

En esta parte realizamos un análisis por variable individual. Tratamos de ir de a poco analizando la información que nos permita responder a la pregunta central del trabajo.

In [ ]:
# 1. Ubicación
print("[1] UBICACION GEOGRAFICA:")
ubicacion_stats = df_analisis.groupby('es_caba')['salario_usd'].agg([
    ('Media', 'mean'), ('Mediana', 'median'), ('Q75', lambda x: x.quantile(0.75)),
    ('Max', 'max'), ('Cantidad', 'count')
]).round(2).sort_values('Mediana', ascending=False)
print(ubicacion_stats)
mejor_ubicacion = ubicacion_stats['Mediana'].idxmax()
print(f"\nGANADOR: {mejor_ubicacion}")
print(f"Mediana: ${ubicacion_stats.loc[mejor_ubicacion, 'Mediana']:,.0f} USD\n")

# 2. Tipo de Contrato
print("[2] TIPO DE CONTRATO:")
contrato_stats = df_analisis.groupby('tipo_contrato')['salario_usd'].agg([
    ('Media', 'mean'), ('Mediana', 'median'), ('Q75', lambda x: x.quantile(0.75)),
    ('Max', 'max'), ('Cantidad', 'count')
]).round(2).sort_values('Mediana', ascending=False)
print(contrato_stats)
mejor_contrato = contrato_stats['Mediana'].idxmax()
print(f"\nGANADOR: {mejor_contrato}")
print(f"Mediana: ${contrato_stats.loc[mejor_contrato, 'Mediana']:,.0f} USD\n")

# 3. Dedicación
print("[3] DEDICACION:")
dedicacion_stats = df_analisis.groupby('dedicacion')['salario_usd'].agg([
    ('Media', 'mean'), ('Mediana', 'median'), ('Q75', lambda x: x.quantile(0.75)),
    ('Max', 'max'), ('Cantidad', 'count')
]).round(2).sort_values('Mediana', ascending=False)
print(dedicacion_stats)
mejor_dedicacion = dedicacion_stats['Mediana'].idxmax()
print(f"\nGANADOR: {mejor_dedicacion}")
print(f"Mediana: ${dedicacion_stats.loc[mejor_dedicacion, 'Mediana']:,.0f} USD\n")

# 4. Modalidad
print("[4] MODALIDAD DE TRABAJO:")
modalidad_stats = df_analisis.groupby('modalidad')['salario_usd'].agg([
    ('Media', 'mean'), ('Mediana', 'median'), ('Q75', lambda x: x.quantile(0.75)),
    ('Max', 'max'), ('Cantidad', 'count')
]).round(2).sort_values('Mediana', ascending=False)
print(modalidad_stats)
mejor_modalidad = modalidad_stats['Mediana'].idxmax()
print(f"\nGANADOR: {mejor_modalidad}")
print(f"Mediana: ${modalidad_stats.loc[mejor_modalidad, 'Mediana']:,.0f} USD\n")

# 5. Seniority
print("[5] SENIORITY:")
seniority_stats = df_analisis.groupby('seniority')['salario_usd'].agg([
    ('Media', 'mean'), ('Mediana', 'median'), ('Q75', lambda x: x.quantile(0.75)),
    ('Max', 'max'), ('Cantidad', 'count')
]).round(2).sort_values('Mediana', ascending=False)
print(seniority_stats)
mejor_seniority = seniority_stats['Mediana'].idxmax()
print(f"\nGANADOR: {mejor_seniority}")
print(f"Mediana: ${seniority_stats.loc[mejor_seniority, 'Mediana']:,.0f} USD\n")

# 6. Puesto
print("[6] PUESTO (Top 10):")
puesto_stats = df_analisis.groupby('puesto')['salario_usd'].agg([
    ('Media', 'mean'), ('Mediana', 'median'), ('Q75', lambda x: x.quantile(0.75)),
    ('Max', 'max'), ('Cantidad', 'count')
]).round(2).sort_values('Mediana', ascending=False).head(10)
print(puesto_stats)
mejor_puesto = puesto_stats['Mediana'].idxmax()
print(f"GANADOR: {mejor_puesto}")
print(f"Mediana: ${puesto_stats.loc[mejor_puesto, 'Mediana']:,.0f} USD\n")

# 7. Forma de Pago
print("[7] FORMA DE PAGO:")
pago_stats = df_analisis.groupby('forma_pago')['salario_usd'].agg([
    ('Media', 'mean'), ('Mediana', 'median'), ('Q75', lambda x: x.quantile(0.75)),
    ('Max', 'max'), ('Cantidad', 'count')
]).round(2).sort_values('Mediana', ascending=False)
print(pago_stats)
mejor_pago = pago_stats['Mediana'].idxmax()
print(f"\nGANADOR: {mejor_pago}")
print(f"Mediana: ${pago_stats.loc[mejor_pago, 'Mediana']:,.0f} USD\n")

# 8. Tamaño de Empresa
print("[8] TAMANO DE EMPRESA:")
empresa_stats = df_analisis.groupby('tamano_empresa')['salario_usd'].agg([
    ('Media', 'mean'), ('Mediana', 'median'), ('Q75', lambda x: x.quantile(0.75)),
    ('Max', 'max'), ('Cantidad', 'count')
]).round(2).sort_values('Mediana', ascending=False)
print(empresa_stats)
mejor_empresa = empresa_stats['Mediana'].idxmax()
print(f"\nGANADOR: {mejor_empresa}")
print(f"Mediana: ${empresa_stats.loc[mejor_empresa, 'Mediana']:,.0f} USD\n")

# 9. Equipo
print("[9] PERSONAS A CARGO:")
equipo_stats = df_analisis.groupby('tiene_equipo')['salario_usd'].agg([
    ('Media', 'mean'), ('Mediana', 'median'), ('Q75', lambda x: x.quantile(0.75)),
    ('Max', 'max'), ('Cantidad', 'count')
]).round(2).sort_values('Mediana', ascending=False)
print(equipo_stats)
mejor_equipo = equipo_stats['Mediana'].idxmax()
print(f"\nGANADOR: {mejor_equipo}")
print(f"Mediana: ${equipo_stats.loc[mejor_equipo, 'Mediana']:,.0f} USD\n")

In [ ]:
# ====================================================================
# 3.2 PERFIL DEL TOP 10%
# ====================================================================
print("-"*70)
print("PERFIL DEL TOP 10% MEJOR PAGADO")
print("-"*70 + "\n")

percentil_90 = df_analisis['salario_usd'].quantile(0.90)
top_10_percent = df_analisis[df_analisis['salario_usd'] >= percentil_90]

print(f"*** Salario minimo Top 10%: ${percentil_90:,.0f} USD")
print(f"*** Trabajadores en Top 10%: {len(top_10_percent)}\n")

print("[Distribucion Top 10%]\n")
print("Ubicacion:")
print(top_10_percent['es_caba'].value_counts(normalize=True).mul(100).round(1))
print("\nTipo de Contrato:")
print(top_10_percent['tipo_contrato'].value_counts(normalize=True).mul(100).round(1))
print("\nSeniority:")
print(top_10_percent['seniority'].value_counts(normalize=True).mul(100).round(1))
print("\nTop 5 Puestos:")
print(top_10_percent['puesto'].value_counts().head(5))
print(f"\nExperiencia promedio: {top_10_percent['experiencia'].mean():.1f} anos")
print(f"Edad promedio: {top_10_percent['edad'].mean():.1f} anos")
print(f"Antiguedad promedio: {top_10_percent['antiguedad'].mean():.1f} anos\n")

De a poco vemos emerger la infomación relevante. Una vez que analizamos las variables anteriores, podemos abordar la cuestión de la "trayectoria profesional" para encontrar el momento del **salto salarial**, _¿suele tardar en llegar?_, _¿llega en algún momento o tiende más a bien a ser estable?_. Esto es lo que intentamos responder a continuación.

In [ ]:
df_analisis['rango_exp'] = pd.cut(
    df_analisis['experiencia'], 
    bins=[0, 2, 5, 8, 12, 100],
    labels=['0-2 años (Junior)', '3-5 años (Semi-Senior)', 
            '6-8 años (Senior)', '9-12 años (Senior+)', '12+ años (Expert)']
)

trayectoria = df_analisis.groupby('rango_exp', observed=True)['salario_usd'].agg([
    ('Cantidad', 'count'), ('Mínimo', 'min'), ('Q25', lambda x: x.quantile(0.25)),
    ('Mediana', 'median'), ('Media', 'mean'), ('Q75', lambda x: x.quantile(0.75)),
    ('Máximo', 'max'), ('Desv_Est', 'std')
]).round(2)

trayectoria['Crecimiento_%'] = trayectoria['Mediana'].pct_change() * 100
trayectoria['Acumulado_%'] = ((trayectoria['Mediana'] / trayectoria['Mediana'].iloc[0]) - 1) * 100

print("EVOLUCIÓN SALARIAL POR EXPERIENCIA:\n")
print(trayectoria)

crecimiento_limpio = trayectoria['Crecimiento_%'].dropna()
if len(crecimiento_limpio) > 0:
    mayor_salto = crecimiento_limpio.idxmax()
    valor_salto = crecimiento_limpio.max()
    print(f"\nMAYOR SALTO SALARIAL: {mayor_salto} (+{valor_salto:.1f}%)\n")

In [ ]:
# ====================================================================
# 4.1 VISUALIZACIONES DE TRAYECTORIA
# ====================================================================
print("[GENERANDO VISUALIZACIONES DE TRAYECTORIA...]\n")

# Visualización: Evolución del salario mediano
fig = go.Figure()

# Área de dispersión (Q25-Q75)
fig.add_trace(go.Scatter(
    x=trayectoria.index.astype(str),
    y=trayectoria['Q75'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    hoverinfo='skip'
))

fig.add_trace(go.Scatter(
    x=trayectoria.index.astype(str),
    y=trayectoria['Q25'],
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(99, 110, 250, 0.2)',
    fill='tonexty',
    name='Rango Q25-Q75',
    hovertemplate='<b>%{x}</b><br>Q25-Q75: $%{y:,.0f}<extra></extra>'
))

# Línea de mediana
fig.add_trace(go.Scatter(
    x=trayectoria.index.astype(str),
    y=trayectoria['Mediana'],
    mode='lines+markers+text',
    name='Salario Mediano',
    line=dict(color='#636EFA', width=3),
    marker=dict(size=12, color='#636EFA'),
    text=trayectoria['Mediana'].round(0),
    texttemplate='$%{text:,.0f}',
    textposition='top center',
    hovertemplate='<b>%{x}</b><br>Mediana: $%{y:,.0f}<extra></extra>'
))

# Línea de media - Comento esta parte del gráfico porque los outliers distorsionan la visualización
# fig.add_trace(go.Scatter(
#     x=trayectoria.index.astype(str),
#     y=trayectoria['Media'],
#     mode='lines+markers',
#     name='Salario Medio',
#     line=dict(color='#00CC96', width=2, dash='dash'),
#     marker=dict(size=8, color='#00CC96'),
#     hovertemplate='<b>%{x}</b><br>Media: $%{y:,.0f}<extra></extra>'
# ))

fig.update_layout(
    title='Trayectoria Salarial: Evolución por Años de Experiencia',
    xaxis_title='Rango de Experiencia',
    yaxis_title='Salario (USD)',
    height=600,
    hovermode='x unified',
    legend=dict(x=0.02, y=0.98)
)
fig.show()

# Gráfico de crecimiento porcentual
fig = go.Figure()

fig.add_trace(go.Bar(
    x=trayectoria.index.astype(str),
    y=trayectoria['Crecimiento_%'],
    name='Crecimiento vs Nivel Anterior',
    marker_color='#636EFA',
    text=trayectoria['Crecimiento_%'].round(1),
    texttemplate='%{text:.1f}%',
    textposition='outside',
    hovertemplate='<b>%{x}</b><br>Crecimiento: %{y:.1f}%<extra></extra>'
))

fig.update_layout(
    title='Crecimiento Salarial entre Niveles de Experiencia',
    xaxis_title='Rango de Experiencia',
    yaxis_title='Crecimiento (%)',
    height=500,
    showlegend=False
)
fig.show()

In [ ]:
# 5. VISUALIZACIONES COMPARATIVAS
print("="*70)
print("VISUALIZACIONES COMPARATIVAS")
print("="*70 + "\n")

# Dashboard Completo
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=('Ubicación', 'Tipo de Contrato', 'Dedicación',
                    'Modalidad', 'Seniority', 'Forma de Pago',
                    'Tamaño Empresa', 'Tiene Equipo', 'Top 5 Puestos'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}]],
    vertical_spacing=0.12,
    horizontal_spacing=0.10
)

# Ubicación
fig.add_trace(go.Bar(
    x=ubicacion_stats.index, 
    y=ubicacion_stats['Mediana'],
    text=ubicacion_stats['Mediana'].round(0),
    texttemplate='$%{text:,.0f}',
    textposition='outside',
    marker_color='#FF6B6B', 
    showlegend=False,
    hovertemplate='<b>%{x}</b><br>Salario: $%{y:,.0f}<extra></extra>'
), row=1, col=1)

# Tipo de Contrato
fig.add_trace(go.Bar(
    x=contrato_stats.index, 
    y=contrato_stats['Mediana'],
    text=contrato_stats['Mediana'].round(0),
    texttemplate='$%{text:,.0f}',
    textposition='outside',
    marker_color='#4ECDC4', 
    showlegend=False,
    hovertemplate='<b>%{x}</b><br>Salario: $%{y:,.0f}<extra></extra>'
), row=1, col=2)

# Dedicación
fig.add_trace(go.Bar(
    x=dedicacion_stats.index, 
    y=dedicacion_stats['Mediana'],
    text=dedicacion_stats['Mediana'].round(0),
    texttemplate='$%{text:,.0f}',
    textposition='outside',
    marker_color='#95E1D3', 
    showlegend=False,
    hovertemplate='<b>%{x}</b><br>Salario: $%{y:,.0f}<extra></extra>'
), row=1, col=3)

# Modalidad
fig.add_trace(go.Bar(
    x=modalidad_stats.index, 
    y=modalidad_stats['Mediana'],
    text=modalidad_stats['Mediana'].round(0),
    texttemplate='$%{text:,.0f}',
    textposition='outside',
    marker_color='#F38181', 
    showlegend=False,
    hovertemplate='<b>%{x}</b><br>Salario: $%{y:,.0f}<extra></extra>'
), row=2, col=1)

# Seniority
fig.add_trace(go.Bar(
    x=seniority_stats.index, 
    y=seniority_stats['Mediana'],
    text=seniority_stats['Mediana'].round(0),
    texttemplate='$%{text:,.0f}',
    textposition='outside',
    marker_color='#AA96DA', 
    showlegend=False,
    hovertemplate='<b>%{x}</b><br>Salario: $%{y:,.0f}<extra></extra>'
), row=2, col=2)

# Forma de Pago
fig.add_trace(go.Bar(
    x=pago_stats.index, 
    y=pago_stats['Mediana'],
    text=pago_stats['Mediana'].round(0),
    texttemplate='$%{text:,.0f}',
    textposition='outside',
    marker_color='#FCBAD3', 
    showlegend=False,
    hovertemplate='<b>%{x}</b><br>Salario: $%{y:,.0f}<extra></extra>'
), row=2, col=3)

# Tamaño Empresa
fig.add_trace(go.Bar(
    x=empresa_stats.index, 
    y=empresa_stats['Mediana'],
    text=empresa_stats['Mediana'].round(0),
    texttemplate='$%{text:,.0f}',
    textposition='outside',
    marker_color='#FFD93D', 
    showlegend=False,
    hovertemplate='<b>%{x}</b><br>Salario: $%{y:,.0f}<extra></extra>'
), row=3, col=1)

# Tiene Equipo
fig.add_trace(go.Bar(
    x=equipo_stats.index, 
    y=equipo_stats['Mediana'],
    text=equipo_stats['Mediana'].round(0),
    texttemplate='$%{text:,.0f}',
    textposition='outside',
    marker_color='#A8D8EA', 
    showlegend=False,
    hovertemplate='<b>%{x}</b><br>Salario: $%{y:,.0f}<extra></extra>'
), row=3, col=2)

# Top 5 Puestos
top_5_puestos = puesto_stats.head(5)
fig.add_trace(go.Bar(
    x=top_5_puestos.index, 
    y=top_5_puestos['Mediana'],
    text=top_5_puestos['Mediana'].round(0),
    texttemplate='$%{text:,.0f}',
    textposition='outside',
    marker_color='#FFAAA5', 
    showlegend=False,
    hovertemplate='<b>%{x}</b><br>Salario: $%{y:,.0f}<extra></extra>'
), row=3, col=3)

fig.update_yaxes(title_text="Salario (USD)")
fig.update_xaxes(tickangle=-45, tickfont=dict(size=9))

fig.update_layout(
    height=1400,
    title_text="Salarios Medianos por Variable Clave",
    title_font_size=16,
    showlegend=False,
    margin=dict(t=100, b=50, l=50, r=50)
)
fig.show()

# Top 10 combinaciones
print("\n[TOP 10 COMBINACIONES]\n")
print("Top 10 Combinaciones (Ubicación + Contrato + Dedicación):")
combinaciones = df_analisis.groupby(['es_caba', 'tipo_contrato', 'dedicacion'])['salario_usd'].agg([
    ('Media', 'mean'),
    ('Mediana', 'median'),
    ('Cantidad', 'count')
]).round(2)

combinaciones_filtradas = combinaciones[combinaciones['Cantidad'] >= 3].sort_values('Mediana', ascending=False).head(10)
print(combinaciones_filtradas)
print()

In [ ]:
# 6. RESUMEN EJECUTIVO
print("RESUMEN EJECUTIVO: LA COMBINACIÓN GANADORA")

resumen = f"""
PERFIL DEL TRABAJADOR IDEAL (MÁXIMOS SALARIOS)

[1] UBICACION: {mejor_ubicacion}
    Mediana: ${ubicacion_stats.loc[mejor_ubicacion, 'Mediana']:,.0f} USD

[2] TIPO CONTRATO: {mejor_contrato}
    Mediana: ${contrato_stats.loc[mejor_contrato, 'Mediana']:,.0f} USD

[3] DEDICACION: {mejor_dedicacion}
    Mediana: ${dedicacion_stats.loc[mejor_dedicacion, 'Mediana']:,.0f} USD

[4] MODALIDAD: {mejor_modalidad}
    Mediana: ${modalidad_stats.loc[mejor_modalidad, 'Mediana']:,.0f} USD

[5] SENIORITY: {mejor_seniority}
    Mediana: ${seniority_stats.loc[mejor_seniority, 'Mediana']:,.0f} USD

[6] PUESTO: {mejor_puesto}
    Mediana: ${puesto_stats.loc[mejor_puesto, 'Mediana']:,.0f} USD

[7] FORMA PAGO: {mejor_pago}
    Mediana: ${pago_stats.loc[mejor_pago, 'Mediana']:,.0f} USD

[8] TAMANO EMPRESA: {mejor_empresa}
    Mediana: ${empresa_stats.loc[mejor_empresa, 'Mediana']:,.0f} USD

[9] LIDERAZGO: {mejor_equipo}
    Mediana: ${equipo_stats.loc[mejor_equipo, 'Mediana']:,.0f} USD

EVOLUCION ESPERADA DEL SALARIO

0-2 años:  ${trayectoria.iloc[0]['Mediana']:,.0f} USD
3-5 años:  ${trayectoria.iloc[1]['Mediana']:,.0f} USD (+{trayectoria.iloc[1]['Crecimiento_%']:.1f}%)
6-8 años:  ${trayectoria.iloc[2]['Mediana']:,.0f} USD (+{trayectoria.iloc[2]['Crecimiento_%']:.1f}%)
9-12 años: ${trayectoria.iloc[3]['Mediana']:,.0f} USD (+{trayectoria.iloc[3]['Crecimiento_%']:.1f}%)
12+ años:  ${trayectoria.iloc[4]['Mediana']:,.0f} USD (+{trayectoria.iloc[4]['Crecimiento_%']:.1f}%)

RECOMENDACIONES PARA MAXIMIZAR SALARIO

1. Ubicarse en {mejor_ubicacion}
2. Buscar contratos tipo {mejor_contrato}
3. Trabajar {mejor_dedicacion}
4. Priorizar modalidad {mejor_modalidad}
5. Desarrollarse hasta {mejor_seniority}
6. Especializarse en roles como {mejor_puesto}
7. Negociar modalidad "{mejor_pago}"
8. Apuntar a empresas {mejor_empresa}
9. Desarrollar capacidad de liderazgo ({mejor_equipo})
10. Acumular ~{top_10_percent['experiencia'].mean():.0f} años de experiencia

Objetivo: Alcanzar Top 10% (${percentil_90:,.0f}+ USD/mes)
"""

print(resumen)

In [ ]:
# 7. MACHINE LEARNING: IMPORTANCIA DE VARIABLES

# Categorizar puestos
def categorizar_puesto(puesto):
    if pd.isna(puesto):
        return 'Other'
    puesto = str(puesto).lower()
    if re.search(r'developer|desarrollador|programador|frontend|backend|fullstack|mobile|web|software engineer', puesto):
        return 'Developer'
    if re.search(r'data|bi |business intelligence|analytics|analista de datos|machine learning|ciencia de datos|scientist', puesto):
        return 'Data & Analytics'
    if re.search(r'manager|management|lider|leader|director|gerente|vp|c-level|head|ceo|cto|cio', puesto):
        return 'Management'
    if re.search(r'devops|sysadmin|infraestructura|infrastructure|sre|networking|cloud|seguridad|security|systems', puesto):
        return 'Infra/Ops/Security'
    if re.search(r'qa|testing|tester|automation|quality', puesto):
        return 'QA / Testing'
    if re.search(r'product|pm|po|product owner|product manager', puesto):
        return 'Product'
    if re.search(r'scrum|agile|coach', puesto):
        return 'Agile / Scrum'
    if re.search(r'ux|ui|designer|disenador|design', puesto):
        return 'UX/UI Design'
    if re.search(r'architect|arquitecto', puesto):
        return 'Architect'
    if re.search(r'consultor|consultant|functional|specialist', puesto):
        return 'Consultant'
    return 'Other'

df_clean['puesto_categoria'] = df_clean['puesto'].apply(categorizar_puesto)

print("[INFO] Categorias de puestos creadas")
print(df_clean['puesto_categoria'].value_counts())
print()

# Visualización: Distribución de categorías de puestos
puesto_cat_counts = df_clean['puesto_categoria'].value_counts()
fig = px.bar(
    x=puesto_cat_counts.index,
    y=puesto_cat_counts.values,
    title='Distribución de Puestos por Categoría',
    labels={'x': 'Categoría', 'y': 'Cantidad'},
    color=puesto_cat_counts.values,
    color_continuous_scale='Viridis'
)
fig.update_layout(height=500, showlegend=False, xaxis_tickangle=-45)
fig.show()

# Simplificar tamaño de empresa
def simplificar_tamano_empresa(tamano):
    if pd.isna(tamano):
        return 'Desconocido'
    if any(x in str(tamano) for x in ['De 2 a', 'De 11', 'De 51', '1 (solamente']):
        return 'Pequeña (1-100)'
    elif any(x in str(tamano) for x in ['De 101', 'De 201', 'De 501']):
        return 'Mediana (101-1000)'
    else:
        return 'Grande (1000+)'

# Aplicar la simplificación al dataframe original
df_clean['tamano_empresa'] = df_clean['tamano_empresa'].apply(simplificar_tamano_empresa)
print(f"[INFO] Tamaño empresa simplificado:")
print(df_clean['tamano_empresa'].value_counts())
print()

# Preparar datos para modelo
features_list = ['experiencia', 'antiguedad', 'edad', 'seniority', 'modalidad',
                 'es_caba', 'tipo_contrato', 'tamano_empresa', 'puesto_categoria',
                 'forma_pago', 'tiene_equipo']

features_list = [col for col in features_list if col in df_clean.columns]
df_model = df_clean[features_list + ['salario_usd']].dropna()

print(f"[INFO] Registros iniciales: {len(df_model)}")

# Eliminación de outliers extremos y valores inválidos
print("\n[LIMPIEZA DE DATOS]")

# 1. Eliminar infinitos y NaN remanentes
df_model = df_model.replace([np.inf, -np.inf], np.nan).dropna()
print(f"Después de eliminar infinitos: {len(df_model)} registros")

# 2. Eliminar salarios negativos o cero
df_model = df_model[df_model['salario_usd'] > 0]
print(f"Después de eliminar salarios <= 0: {len(df_model)} registros")

# 3. Filtrar outliers usando IQR (rango intercuartílico)
Q1 = df_model['salario_usd'].quantile(0.005)  # Percentil 0.5
Q99 = df_model['salario_usd'].quantile(0.995)  # Percentil 99.5 

print(f"Rango válido: ${Q1:,.0f} - ${Q99:,.0f} USD")

df_model = df_model[(df_model['salario_usd'] >= Q1) & (df_model['salario_usd'] <= Q99)]
print(f"Después de filtrar outliers extremos: {len(df_model)} registros")

# 4. Verificar rangos razonables en otras variables
df_model = df_model[
    (df_model['experiencia'] >= 0) & (df_model['experiencia'] <= 50) &
    (df_model['edad'] >= 18) & (df_model['edad'] <= 80) &
    (df_model['antiguedad'] >= 0) & (df_model['antiguedad'] <= 50)
]
print(f"Después de validar rangos de variables: {len(df_model)} registros")

print(f"\n[INFO] Registros finales para modelo: {len(df_model)}")
print(f"[INFO] Variables predictoras: {len(features_list)}")
print(f"[INFO] Salario USD - Min: ${df_model['salario_usd'].min():,.0f} | Max: ${df_model['salario_usd'].max():,.0f}")
print(f"[INFO] Salario USD - Media: ${df_model['salario_usd'].mean():,.0f} | Mediana: ${df_model['salario_usd'].median():,.0f}\n")

# Encoding
X = pd.get_dummies(df_model[features_list], drop_first=True)
y = df_model['salario_usd']
feature_names = X.columns.tolist()

print(f"[INFO] Features después de encoding: {len(feature_names)}")

# Asegurar que todas las columnas sean numéricas
X = X.apply(pd.to_numeric, errors='coerce')

# Verificar que no haya infinitos o NaN en X o y
nan_count_x = X.isna().sum().sum()
print(f"[VALIDACIÓN] NaN en X: {nan_count_x}")

if nan_count_x > 0:
    print(f"Eliminando {nan_count_x} valores NaN generados en encoding...")
    X = X.fillna(0)  # Rellenar NaN con 0 (seguro para variables dummy)

# Verificar infinitos solo en columnas numéricas
try:
    inf_count_x = np.isinf(X.select_dtypes(include=[np.number]).values).sum()
    print(f"[VALIDACIÓN] Infinitos en X: {inf_count_x}")
except:
    print(f"[VALIDACIÓN] Infinitos en X: 0 (no se pudo verificar)")

print(f"[VALIDACIÓN] NaN en y: {y.isna().sum()}")
print(f"[VALIDACIÓN] Infinitos en y: {np.isinf(y.values).sum()}\n")

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"[MODELO] Entrenamiento: {len(X_train)} | Test: {len(X_test)}")

# Entrenar
model = RandomForestRegressor(
    n_estimators=50,           
    max_depth=8,               
    min_samples_split=20,      
    min_samples_leaf=10,       
    max_features='sqrt',       
    max_samples=0.8,           
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)
print("[MODELO] Entrenado exitosamente\n")

# Métricas
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print("MÉTRICAS DEL MODELO")
print(f"MAE Train: ${mae_train:,.0f} USD")
print(f"MAE Test:  ${mae_test:,.0f} USD")
print(f"RMSE Test: ${rmse_test:,.0f} USD")
print(f"R² Train:  {r2_train:.4f} ({r2_train*100:.2f}%)")
print(f"R² Test:   {r2_test:.4f} ({r2_test*100:.2f}%)")

# Verificar overfitting
diff_r2 = r2_train - r2_test
if diff_r2 > 0.1:
    print(f"\nAdvertencia: Posible overfitting (diferencia R²: {diff_r2:.3f})")
else:
    print(f"\nEl modelo generaliza bien (diferencia R²: {diff_r2:.3f})")
print()

print("VALIDACIÓN CRUZADA (5-FOLD):")
cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2', n_jobs=-1)
print(f"R² promedio: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
print(f"R² por fold: {[f'{score:.3f}' for score in cv_scores]}")

# Evaluar también MAE con validación cruzada
cv_mae = -cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
print(f"MAE promedio: ${cv_mae.mean():,.0f} (±${cv_mae.std():,.0f})")
print()

In [ ]:
# 7.1 VISUALIZACIÓN: PREDICCIONES VS REALES

# Scatter plot con muestreo si hay muchos puntos
n_points = len(y_test)
if n_points > 500:
    sample_indices = np.random.choice(n_points, 500, replace=False)
    y_test_sample = y_test.iloc[sample_indices]
    y_pred_sample = y_pred_test[sample_indices]
else:
    y_test_sample = y_test
    y_pred_sample = y_pred_test

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=y_test_sample,
    y=y_pred_sample,
    mode='markers',
    marker=dict(
        color=y_test_sample,
        colorscale='Viridis',
        size=6,
        opacity=0.5,
        colorbar=dict(title="Salario Real (USD)", len=0.7),
        line=dict(width=0.5, color='white')
    ),
    name='Predicciones',
    hovertemplate='<b>Real:</b> $%{x:,.0f}<br><b>Predicho:</b> $%{y:,.0f}<extra></extra>'
))

# Línea de predicción perfecta
min_val = min(y_test.min(), y_pred_test.min())
max_val = max(y_test.max(), y_pred_test.max())
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode='lines',
    line=dict(color='red', dash='dash', width=2),
    name='Predicción Perfecta',
    hoverinfo='skip'
))

# Agregar bandas de error (±MAE)
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val + mae_test, max_val + mae_test],
    mode='lines',
    line=dict(color='orange', dash='dot', width=1),
    name=f'Error +${mae_test:,.0f}',
    hoverinfo='skip'
))

fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val - mae_test, max_val - mae_test],
    mode='lines',
    line=dict(color='orange', dash='dot', width=1),
    name=f'Error -${mae_test:,.0f}',
    hoverinfo='skip'
))

fig.update_layout(
    title=dict(
        text=f'Predicciones vs Valores Reales<br><sub>R² = {r2_test:.3f} | MAE = ${mae_test:,.0f} USD</sub>',
        x=0.5,
        xanchor='center'
    ),
    xaxis=dict(
        title='Salario Real (USD)',
        tickformat='$,.0f',
        gridcolor='lightgray'
    ),
    yaxis=dict(
        title='Salario Predicho (USD)',
        tickformat='$,.0f',
        gridcolor='lightgray'
    ),
    height=700,
    hovermode='closest',
    plot_bgcolor='white',
    legend=dict(
        orientation="v",
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255,255,255,0.8)"
    ),
    margin=dict(t=100, b=50, l=80, r=50)
)

if n_points > 500:
    fig.add_annotation(
        text=f"Nota: Mostrando 500 de {n_points} puntos para mejor visualización",
        xref="paper", yref="paper",
        x=0.5, y=-0.12,
        showarrow=False,
        font=dict(size=10, color="gray")
    )

fig.show()

In [ ]:
# IMPORTANCIA DE VARIABLES

# Calcular importancias
importances = model.feature_importances_
feature_imp_df = pd.DataFrame({
    'Variable': feature_names,
    'Importancia_%': importances * 100
}).sort_values(by='Importancia_%', ascending=False)

print("TOP 15 VARIABLES MÁS IMPORTANTES:\n")
print(feature_imp_df.head(15))
print()

# Agrupar por categoría
def extraer_categoria(var):
    if 'seniority_' in var: return 'Seniority'
    elif 'puesto_categoria_' in var: return 'Puesto/Rol'
    elif 'modalidad_' in var: return 'Modalidad'
    elif 'tipo_contrato_' in var: return 'Tipo Contrato'
    elif 'tamano_empresa_' in var: return 'Tamano Empresa'
    elif 'forma_pago_' in var: return 'Forma de Pago'
    elif 'es_caba_' in var: return 'Ubicacion'
    elif 'tiene_equipo_' in var: return 'Liderazgo'
    elif var in ['experiencia', 'antiguedad', 'edad']: return var.capitalize()
    else: return 'Otros'

feature_imp_df['Categoria'] = feature_imp_df['Variable'].apply(extraer_categoria)
imp_agrupada = feature_imp_df.groupby('Categoria')['Importancia_%'].sum().sort_values(ascending=False)

print("IMPORTANCIA POR CATEGORÍA:\n")
print(imp_agrupada.to_frame())
print()

# Visualización: Top 20 Variables
fig = px.bar(
    feature_imp_df.head(20).sort_values(by='Importancia_%', ascending=True),
    x='Importancia_%',
    y='Variable',
    title='Top 20 Variables Más Importantes para Predecir el Salario',
    labels={'Importancia_%': 'Importancia (%)', 'Variable': 'Variable'},
    text='Importancia_%',
    color='Importancia_%',
    color_continuous_scale='Viridis'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(height=800, showlegend=False)
fig.show()

# Visualización: Importancia Agrupada
fig = px.bar(
    x=imp_agrupada.values,
    y=imp_agrupada.index,
    orientation='h',
    title='Importancia Agrupada por Categoría de Variable',
    labels={'x': 'Importancia Total (%)', 'y': 'Categoría'},
    text=imp_agrupada.values,
    color=imp_agrupada.values,
    color_continuous_scale='Plasma'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(height=600, showlegend=False)
fig.show()

In [ ]:
# 7.3 ANÁLISIS DE RESIDUOS

# Calcular residuos
residuos = y_test - y_pred_test

print("ESTADISTICAS DE LOS RESIDUOS:")
print(f"Media: ${residuos.mean():,.0f} USD")
print(f"Mediana: ${residuos.median():,.0f} USD")
print(f"Desviación Estándar: ${residuos.std():,.0f} USD\n")

# Distribución de residuos
fig = px.histogram(
    residuos,
    nbins=50,
    title='Distribución de Errores del Modelo',
    labels={'value': 'Error (USD)', 'count': 'Frecuencia'},
    color_discrete_sequence=['#636EFA']
)
fig.add_vline(x=0, line_dash="dash", line_color="red", annotation_text="Error = 0")
fig.update_layout(height=500, showlegend=False)
fig.show()

In [ ]:
# 8. CONCLUSIONES FINALES

top_cat = imp_agrupada.index[0]
top_val = imp_agrupada.iloc[0]

print("CONCLUSIONES DEL MODELO PREDICTIVO")

print(f"""
El modelo Random Forest revela que:

* Precision (R² test): {r2_test:.1%} - Explica {r2_test*100:.1f}% de la variabilidad salarial
* Precision (R² CV): {cv_scores.mean():.1%} - Validación cruzada confirma estabilidad
* Error promedio: ${mae_test:,.0f} USD/mes (±${cv_mae.std():,.0f})
* Overfitting: {'✓ Controlado' if diff_r2 < 0.2 else '⚠ Presente'} (diferencia R²: {diff_r2:.3f})
* Factor mas determinante: {top_cat} ({top_val:.1f}% de importancia)

Top 3 variables individuales:
1. {feature_imp_df.iloc[0]['Variable']}: {feature_imp_df.iloc[0]['Importancia_%']:.2f}%
2. {feature_imp_df.iloc[1]['Variable']}: {feature_imp_df.iloc[1]['Importancia_%']:.2f}%
3. {feature_imp_df.iloc[2]['Variable']}: {feature_imp_df.iloc[2]['Importancia_%']:.2f}%

Implicación práctica: Para maximizar salario, el trabajador debe enfocarse en optimizar su
{top_cat.lower()}, seguido de las variables identificadas arriba.
""")